<a href="https://colab.research.google.com/github/ferreirabruno7/Text_analyze/blob/master/previs%C3%A3o_cluster_texto_covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#acessar google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#importação de bibliotecas
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score

In [0]:
#carregar dataset
comentarios = pd.read_csv("/content/drive/My Drive/Colab Notebooks/covid_19_cluuster_new.csv", delimiter = ',', encoding='utf-8')
comentarios.head()

,Unnamed: 0,protocolo,data_manifestacao,municipio_denunciante,municipio_fato,localFato,manifestacao,status_atual,desc_status,sigla,nome,tipo_manifestacao,assunto,sub_assunto,tokens,cluster,coseno
0,0,2020.0314.215152-35,2020-03-14 21:51:52,NaN,GOIANIA,Setor Candida de Morais,Elogio com sugestão de ações,55,Finalizado pela ouvidoria do Órgão.,SGG/GOVERNADORIA,Secretaria Geral da Governadoria,Elogio,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...","['Elogio', 'com', 'sugestão', 'de', 'ações']",3,0.00000
1,1,2020.0316.203054-88,2020-03-16 20:30:54,NaN,GOIANIA,Vila Redenção,Na empresa BTCC (prestadora de Serviços para O...,55,Finalizado pela ouvidoria do Órgão.,SES,Secretaria de Estado da Saúde,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...","['Na', 'empresa', 'BTCC', '(', 'prestadora', '...",5,0.00000
2,2,2020.0318.081504-41,2020-03-18 08:15:04,NaN,ANAPOLIS,Vapt Vupt,Considerando o decreto do Sr. Governador quant...,55,Finalizado pela ouvidoria do Órgão.,SEAD,Secretaria de Estado de Administração (Antiga ...,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...","['Considerando', 'o', 'decreto', 'do', 'Sr', '...",5,0.03495
3,3,2020.0318.115152-35,2020-03-18 11:51:52,NaN,GOIANIA,POLÍCIA CÍVIL,Em meio a grave pandemia de coronavirus pela q...,55,Finalizado pela ouvidoria do Órgão.,SSP,Sec. Estado da Segurança Pública (Policias Civ...,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...","['Em', 'meio', 'a', 'grave', 'pandemia', 'de',...",5,0.00000
4,4,2020.0318.115845-69,2020-03-18 11:58:45,NaN,GOIANIA,NaN,Em virtude do Decreto emitido pelo Governador ...,55,Finalizado pela ouvidoria do Órgão.,SSP,Sec. Estado da Segurança Pública (Policias Civ...,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...","['Em', 'virtude', 'do', 'Decreto', 'emitido', ...",5,0.00000


In [0]:
comentarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             2129 non-null   int64  
 1   protocolo              2129 non-null   object 
 2   data_manifestacao      2129 non-null   object 
 3   municipio_denunciante  277 non-null    object 
 4   municipio_fato         2129 non-null   object 
 5   localFato              1841 non-null   object 
 6   manifestacao           2129 non-null   object 
 7   status_atual           2129 non-null   int64  
 8   desc_status            340 non-null    object 
 9   sigla                  2129 non-null   object 
 10  nome                   2129 non-null   object 
 11  tipo_manifestacao      2129 non-null   object 
 12  assunto                2129 non-null   object 
 13  sub_assunto            2129 non-null   object 
 14  tokens                 2129 non-null   object 
 15  clus

In [0]:
del comentarios['Unnamed: 0']
del comentarios['tokens']
del comentarios['coseno']


In [0]:
comentarios.describe()

,status_atual,cluster
count,2129.000000,2129.000000
mean,5.827619,3.833255
std,12.599561,1.671832
min,1.000000,0.000000
25%,1.000000,3.000000
50%,1.000000,4.000000
75%,1.000000,5.000000
max,55.000000,7.000000


In [0]:
comentarios['cluster'].value_counts()

3    552
5    509
2    370
6    216
4    203
7    141
1    125
0     13
Name: cluster, dtype: int64

In [0]:
comentarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   protocolo              2129 non-null   object
 1   data_manifestacao      2129 non-null   object
 2   municipio_denunciante  277 non-null    object
 3   municipio_fato         2129 non-null   object
 4   localFato              1841 non-null   object
 5   manifestacao           2129 non-null   object
 6   status_atual           2129 non-null   int64 
 7   desc_status            340 non-null    object
 8   sigla                  2129 non-null   object
 9   nome                   2129 non-null   object
 10  tipo_manifestacao      2129 non-null   object
 11  assunto                2129 non-null   object
 12  sub_assunto            2129 non-null   object
 13  cluster                2129 non-null   int64 
dtypes: int64(2), object(12)
memory usage: 233.0+ KB


In [0]:
#funcão de processaento de textos
def pre_process(texto):
  texto = texto.replace(' nan ','')
  texto = texto.replace('nan ','')
  texto = texto.replace(' nan','')
  texto = re.sub('\r','',texto)
  texto = re.sub('\n','',texto)
  texto = re.sub(',',' ',texto)
  texto = re.sub('\.',' ',texto)
  texto = re.sub('\?',' ',texto)
  texto = re.sub('\!',' ',texto)
  texto = re.sub('\;',' ',texto)
  return texto

In [0]:
#aplicar função de limpeza na coluna
comentarios['new_manifestacao'] = comentarios.manifestacao.apply(pre_process)

In [0]:
#definindo variável de X e Y
texto = comentarios['new_manifestacao'].values
classes = comentarios['cluster'].values

In [0]:
## Definido o corpus

corpus = comentarios.new_manifestacao.values
corpus

array(['Elogio com sugestão de ações ',
       'Na empresa BTCC (prestadora de Serviços para OI) não esta havendo nenhuma ação (alteração/prevenção) visando evitar a possível propagação do CORONAVIRUS entre seus colaboradores (funcionários)  haja visto que a empresa possui quase 10 mil funcionários em plena atividade em Goiânia  cujo contato e aglomeração favorecem esse tipo de disseminação tão combatida nesse momento de crise Este fato se estende as demais Empresas Operadores de Telemarketing que  mesmo possuindo quantidades menores de funcionários  se tornam possíveis focos de propagação do Covid-19  ',
       'Considerando o decreto do Sr  Governador quanto às medidas de enfrentamento ao Coronavírus  quero fazer minha reclamação acerca dos Vaptvutps:Estão funcionando normalmente e pretendem funcionar a despeito do decreto  inclusive em Shopping Centers Os gestores alegam que os atendimentos estão sendo por agendamento  e realmente estão sendo assim  Porém  a cada 30 minutos são disp

In [0]:
## Vetorizando o corpus (e criando nosso X)

cvec = CountVectorizer(ngram_range=(2,2),min_df=1)
x = cvec.fit_transform(corpus)
x

<2129x42930 sparse matrix of type '<class 'numpy.int64'>'
	with 79920 stored elements in Compressed Sparse Row format>

In [0]:
cvec.vocabulary_

{'elogio com': 13778,
 'com sugestão': 7653,
 'sugestão de': 38251,
 'de ações': 10077,
 'na empresa': 26260,
 'empresa btcc': 14352,
 'btcc prestadora': 5470,
 'prestadora de': 32656,
 'de serviços': 10924,
 'serviços para': 37435,
 'para oi': 29825,
 'oi não': 28644,
 'não esta': 27966,
 'esta havendo': 15746,
 'havendo nenhuma': 20925,
 'nenhuma ação': 26812,
 'ação alteração': 4784,
 'alteração prevenção': 2706,
 'prevenção visando': 32704,
 'visando evitar': 42252,
 'evitar possível': 16774,
 'possível propagação': 32205,
 'propagação do': 33000,
 'do coronavirus': 12805,
 'coronavirus entre': 9047,
 'entre seus': 15118,
 'seus colaboradores': 37594,
 'colaboradores funcionários': 7112,
 'funcionários haja': 19544,
 'haja visto': 20907,
 'visto que': 42297,
 'que empresa': 33992,
 'empresa possui': 14617,
 'possui quase': 32183,
 'quase 10': 33800,
 '10 mil': 191,
 'mil funcionários': 25378,
 'funcionários em': 19484,
 'em plena': 14047,
 'plena atividade': 31262,
 'atividade em':

In [0]:
## Definindo Y

y = np.array(comentarios['cluster'].values)
y

array([3, 5, 5, ..., 5, 2, 2])

In [0]:
#vetorizando dados
vectorizer = CountVectorizer()
freq_text = vectorizer.fit(texto)


In [0]:
freq_text = vectorizer.transform(texto)


Modelo Multinomial

In [0]:
modelo = MultinomialNB()
modelo.fit(freq_text,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
testes = ['Temos que fechar as indústrias, somos todos vetores de uma possível transmissão']

In [0]:
freq_testes = vectorizer.transform(testes)
modelo.predict(freq_testes)

array([5])

In [0]:
resultados = cross_val_predict(modelo, freq_text, classes, cv=10)


In [0]:
metrics.accuracy_score(classes,resultados)


0.565054015969939

In [0]:
print(classification_report(resultados, classes))

              precision    recall  f1-score   support

           0       0.69      1.00      0.82         9
           1       0.13      0.89      0.22        18
           2       0.61      0.58      0.59       389
           3       0.56      0.67      0.61       456
           4       0.26      0.79      0.39        67
           5       0.98      0.46      0.63      1079
           6       0.32      0.80      0.46        86
           7       0.18      1.00      0.30        25

    accuracy                           0.57      2129
   macro avg       0.47      0.77      0.50      2129
weighted avg       0.75      0.57      0.60      2129



Modelo Gaussian

In [0]:
from sklearn.naive_bayes import GaussianNB
classificador = GaussianNB()

In [0]:
X = freq_text.todense()


In [0]:
classificador.fit(X,y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
testes2 = ['Temos que fechar as indústrias, somos todos vetores de uma possível transmissão']
freq_testes2 = vectorizer.transform(testes2)


In [0]:
X_teste = freq_testes2.todense()


In [0]:
freq_testes_gaussian = vectorizer.transform(testes2)
classificador.predict(X_teste)

array([5])

In [0]:
resultados = cross_val_predict(classificador, X, classes, cv=10)


In [0]:
metrics.accuracy_score(classes,resultados)


0.3771723813997182

In [0]:
print(classification_report(resultados, classes))

              precision    recall  f1-score   support

           0       0.85      1.00      0.92        11
           1       0.21      0.27      0.23        98
           2       0.25      0.33      0.28       278
           3       0.31      0.41      0.35       422
           4       0.15      0.29      0.20       108
           5       0.85      0.41      0.55      1058
           6       0.13      0.27      0.18       104
           7       0.08      0.22      0.12        50

    accuracy                           0.38      2129
   macro avg       0.35      0.40      0.35      2129
weighted avg       0.55      0.38      0.42      2129



Arvore de Decisão

In [0]:
from sklearn.tree import DecisionTreeClassifier

In [0]:
dtree = DecisionTreeClassifier()

In [0]:
dtree.fit(X,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [0]:
testes3 = ['Temos que fechar as indústrias, somos todos vetores de uma possível transmissão']
freq_testes3 = vectorizer.transform(testes3)


In [0]:
dtree.predict(freq_testes3)

array([3])

In [0]:
resultados = cross_val_predict(dtree, X, classes, cv=10)


In [0]:
metrics.accuracy_score(classes,resultados)


0.669798027242837

In [0]:
print(classification_report(resultados, classes))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85        13
           1       0.63      0.61      0.62       129
           2       0.59      0.61      0.60       356
           3       0.67      0.68      0.68       538
           4       0.65      0.64      0.65       206
           5       0.73      0.72      0.73       512
           6       0.65      0.64      0.64       220
           7       0.76      0.69      0.72       155

    accuracy                           0.67      2129
   macro avg       0.69      0.68      0.69      2129
weighted avg       0.67      0.67      0.67      2129



Floresta Aleatória

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
rcf = RandomForestClassifier(n_estimators=500)

In [0]:
rcf.fit(X,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
testes4 = ['Temos que fechar as indústrias, somos todos vetores de uma possível transmissão']
freq_testes4 = vectorizer.transform(testes4)

In [0]:
rcf.predict(freq_testes4)

array([3])

In [0]:
resultados = cross_val_predict(rcf, X, classes, cv=10)


In [0]:
metrics.accuracy_score(classes,resultados)


0.7956787224048849

In [0]:
print(classification_report(resultados, classes))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96        12
           1       0.54      0.89      0.68        76
           2       0.82      0.73      0.78       415
           3       0.79      0.80      0.79       549
           4       0.77      0.88      0.82       178
           5       0.90      0.77      0.83       595
           6       0.74      0.86      0.80       186
           7       0.70      0.83      0.76       118

    accuracy                           0.80      2129
   macro avg       0.77      0.85      0.80      2129
weighted avg       0.81      0.80      0.80      2129



Carregando dados de teste

In [0]:
#carregar dados de testes
comentarios_teste = pd.read_csv("/content/drive/My Drive/Colab Notebooks/manifestacao_15.csv", delimiter = ';', encoding='iso-8859-1')
comentarios_teste.head()

,protocolo,data_manifestacao,municipio_denunciante,municipio_fato,localFato,manifestacao,status_atual,desc_status,sigla,nome,tipo_manifestacao,assunto,sub_assunto
0,2020.0304.134443-14,2020-03-04 13:44:43,NaN,GOIANIA,Setor campinas,A igreja congregação cristã no brasil faz uso ...,55,Finalizado pela ouvidoria do Órgão.,SES,Secretaria de Estado da Saúde,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform..."
1,2020.0313.090346-19,2020-03-13 09:03:46,GOIANIA,GOIANIA,"Rua Sergipe, Qd. B, Lt. 8, Bl-7, Apto. 301, Se...",ILUSTRÍSSIMO (A) SENHOR (A) SUPERINTENDENTE DO...,55,Finalizado pela ouvidoria do Órgão.,SEAD,Secretaria de Estado de Administração (Antiga ...,Solicitação,COVID-19 - Corona Vírus,Informações
2,2020.0313.120416-17,2020-03-13 12:04:16,NaN,ANAPOLIS,ANIMA CENTRO HOSPITALAR LTDA CNPJ 18034143000180,Venho por meu deste realizar a denuncia de que...,55,Finalizado pela ouvidoria do Órgão.,SES,Secretaria de Estado da Saúde,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform..."
3,2020.0314.132210-80,2020-03-14 13:22:10,NaN,GOIANIA,Cemac Juarez barbosa,Alguém ligado ao governador Ronaldo Caiado pod...,55,Finalizado pela ouvidoria do Órgão.,SES,Secretaria de Estado da Saúde,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform..."
4,2020.0314.215152-35,2020-03-14 21:51:52,NaN,GOIANIA,Setor Candida de Morais,Elogio com sugestão de ações,55,Finalizado pela ouvidoria do Órgão.,SGG/GOVERNADORIA,Secretaria Geral da Governadoria,Elogio,Saúde,"Corona Vírus - Denúncia, solicitação ou inform..."


In [0]:
comentarios_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4203 entries, 0 to 4202
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   protocolo              4203 non-null   object
 1   data_manifestacao      4203 non-null   object
 2   municipio_denunciante  538 non-null    object
 3   municipio_fato         4203 non-null   object
 4   localFato              3617 non-null   object
 5   manifestacao           4203 non-null   object
 6   status_atual           4203 non-null   int64 
 7   desc_status            3732 non-null   object
 8   sigla                  4203 non-null   object
 9   nome                   4203 non-null   object
 10  tipo_manifestacao      4203 non-null   object
 11  assunto                4203 non-null   object
 12  sub_assunto            4203 non-null   object
dtypes: int64(1), object(12)
memory usage: 427.0+ KB


In [0]:
comentarios_teste['new_manifestacao'] = comentarios_teste.manifestacao.apply(pre_process)

In [0]:
texto_2 = comentarios_teste['new_manifestacao'].values


In [0]:
corpus_2 = comentarios_teste.new_manifestacao.values
corpus_2

array(['A igreja congregação cristã no brasil faz uso de copo coletivo nos dias de santa ceia ou seja se em uma igreja têm 500membros são 500pessoas tomando vinho no mesmo copo   isso pra eles é comum mais pode ser transmitidas vários tipos de doenças nos seus respectivos membros   por favor tomem alguma providência porque estamos diante de uma catástrofe anunciada   ',
       'ILUSTRÍSSIMO (A) SENHOR (A) SUPERINTENDENTE DOS VAPT-VUPT\'S DA SECRETARIA DE ADMINISTRAÇÃO DO ESTADO DE GOIÁS URGENTÍSSIMO    Assunto: Coronavírus (Sars-Cov-2) - Doença Covid-19Órgão Estatal: Unidades de Atendimentos dos Vapt-Vupt\'s da SEAD/GO Goiânia  13 de março de 2020  MARCELO SANTOS DE OLIVEIRA  brasileiro  solteiro  funcionário público efetivo e advogado inscrito na OAB/GO 35 340  portador do RG 4371122 da SSP/GO  CPF 007 086 881-60  residente e domiciliado na Rua Sergipe  Qd  B  Lt  8  Bl-7  Apto  301  Setor Urias Magalhães  Goiânia - GO  CEP: 74 565-220  e-mail: marceloassjur@hotmail com  vem  à presen

In [0]:
corpus_2.reshape(1, -1)

array([['A igreja congregação cristã no brasil faz uso de copo coletivo nos dias de santa ceia ou seja se em uma igreja têm 500membros são 500pessoas tomando vinho no mesmo copo   isso pra eles é comum mais pode ser transmitidas vários tipos de doenças nos seus respectivos membros   por favor tomem alguma providência porque estamos diante de uma catástrofe anunciada   ',
        'ILUSTRÍSSIMO (A) SENHOR (A) SUPERINTENDENTE DOS VAPT-VUPT\'S DA SECRETARIA DE ADMINISTRAÇÃO DO ESTADO DE GOIÁS URGENTÍSSIMO    Assunto: Coronavírus (Sars-Cov-2) - Doença Covid-19Órgão Estatal: Unidades de Atendimentos dos Vapt-Vupt\'s da SEAD/GO Goiânia  13 de março de 2020  MARCELO SANTOS DE OLIVEIRA  brasileiro  solteiro  funcionário público efetivo e advogado inscrito na OAB/GO 35 340  portador do RG 4371122 da SSP/GO  CPF 007 086 881-60  residente e domiciliado na Rua Sergipe  Qd  B  Lt  8  Bl-7  Apto  301  Setor Urias Magalhães  Goiânia - GO  CEP: 74 565-220  e-mail: marceloassjur@hotmail com  vem  à pres

In [0]:
cvec_teste = CountVectorizer(vocabulary=vectorizer.vocabulary)
x_teste = cvec_teste.fit_transform(corpus_2)
x_teste.reshape(1, -1)

<1x59993622 sparse matrix of type '<class 'numpy.int64'>'
	with 150414 stored elements in COOrdinate format>

In [0]:
cvec_teste.vocabulary_

{'igreja': 7526,
 'congregação': 3877,
 'cristã': 4348,
 'no': 9602,
 'brasil': 2743,
 'faz': 6420,
 'uso': 13653,
 'de': 4511,
 'copo': 4188,
 'coletivo': 3562,
 'nos': 9642,
 'dias': 5015,
 'santa': 12283,
 'ceia': 3204,
 'ou': 10002,
 'seja': 12386,
 'se': 12325,
 'em': 5492,
 'uma': 13562,
 'têm': 13535,
 '500membros': 701,
 'são': 12925,
 '500pessoas': 702,
 'tomando': 13261,
 'vinho': 13964,
 'mesmo': 9108,
 'isso': 8080,
 'pra': 10824,
 'eles': 5446,
 'comum': 3744,
 'mais': 8750,
 'pode': 10638,
 'ser': 12463,
 'transmitidas': 13398,
 'vários': 14118,
 'tipos': 13213,
 'doenças': 5262,
 'seus': 12521,
 'respectivos': 11900,
 'membros': 9049,
 'por': 10709,
 'favor': 6408,
 'tomem': 13273,
 'alguma': 1632,
 'providência': 11233,
 'porque': 10714,
 'estamos': 6022,
 'diante': 5009,
 'catástrofe': 3161,
 'anunciada': 1901,
 'ilustríssimo': 7544,
 'senhor': 12424,
 'superintendente': 12845,
 'dos': 5297,
 'vapt': 13724,
 'vupt': 14110,
 'da': 4469,
 'secretaria': 12334,
 'administr

In [0]:
cvec_teste

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [0]:
vectorizer_2 = CountVectorizer(analyzer="word")
freq_text_ = vectorizer.transform(textso_2)

In [0]:
vectorizer_2

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [0]:
#previsao
pred = modelo.predict_proba(freq_text_)
pred

array([[2.21293782e-040, 3.98839791e-024, 3.16390273e-012, ...,
        1.00000000e+000, 8.13597744e-021, 4.17020551e-022],
       [0.00000000e+000, 5.20003181e-232, 2.43679605e-144, ...,
        1.00000000e+000, 5.84317949e-137, 4.35311320e-224],
       [3.63388266e-107, 6.12358967e-048, 6.03362343e-018, ...,
        1.00000000e+000, 3.07695333e-031, 8.23963450e-048],
       ...,
       [1.29902212e-015, 6.10359778e-006, 9.02468050e-001, ...,
        1.49375529e-002, 1.17784593e-004, 8.62072332e-006],
       [1.59437946e-045, 1.12483637e-014, 8.56009003e-005, ...,
        9.99914399e-001, 1.25387258e-013, 2.36427354e-018],
       [1.63398215e-036, 1.96104898e-010, 1.52935916e-006, ...,
        9.97040750e-001, 2.91668675e-003, 8.45423159e-010]])

In [0]:

previsageral = pd.DataFrame(pred)

In [0]:
previsageral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4203 entries, 0 to 4202
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       4203 non-null   float64
 1   1       4203 non-null   float64
 2   2       4203 non-null   float64
 3   3       4203 non-null   float64
 4   4       4203 non-null   float64
 5   5       4203 non-null   float64
 6   6       4203 non-null   float64
 7   7       4203 non-null   float64
dtypes: float64(8)
memory usage: 262.8 KB


In [0]:
previsageral.round(3)

,0,1,2,3,4,5,6,7
0,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
1,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
2,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
3,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
4,0.0,0.01,0.091,0.632,0.172,0.029,0.042,0.023
...,...,...,...,...,...,...,...,...
4198,0.0,0.00,0.000,0.000,0.000,0.999,0.001,0.000
4199,0.0,0.00,0.006,0.000,0.000,0.994,0.000,0.000
4200,0.0,0.00,0.902,0.082,0.000,0.015,0.000,0.000
4201,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000


In [0]:
previsageral.round(3)

,0,1,2,3,4,5,6,7
0,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
1,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
2,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
3,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
4,0.0,0.01,0.091,0.632,0.172,0.029,0.042,0.023
...,...,...,...,...,...,...,...,...
4198,0.0,0.00,0.000,0.000,0.000,0.999,0.001,0.000
4199,0.0,0.00,0.006,0.000,0.000,0.994,0.000,0.000
4200,0.0,0.00,0.902,0.082,0.000,0.015,0.000,0.000
4201,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000


In [0]:
previsageral

,0,1,2,3,4,5,6,7
0,2.212938e-40,3.988398e-24,3.163903e-12,3.700342e-10,4.405010e-20,1.000000,8.135977e-21,4.170206e-22
1,0.000000e+00,5.200032e-232,2.436796e-144,8.940639e-119,1.092048e-187,1.000000,5.843179e-137,4.353113e-224
2,3.633883e-107,6.123590e-48,6.033623e-18,3.187320e-17,9.259340e-43,1.000000,3.076953e-31,8.239635e-48
3,2.350659e-31,6.371644e-19,7.884802e-08,1.597959e-09,8.010764e-15,1.000000,5.186319e-12,1.916474e-14
4,2.451224e-04,1.009735e-02,9.134820e-02,6.319433e-01,1.721640e-01,0.029046,4.181769e-02,2.333882e-02
...,...,...,...,...,...,...,...,...
4198,1.603017e-25,5.285910e-10,4.951411e-07,1.012685e-06,3.112807e-09,0.998505,1.493793e-03,6.600633e-12
4199,9.183536e-40,2.741822e-14,5.557950e-03,2.358841e-05,1.889650e-05,0.994382,1.725913e-05,2.994844e-12
4200,1.299022e-15,6.103598e-06,9.024681e-01,8.246019e-02,1.702679e-06,0.014938,1.177846e-04,8.620723e-06
4201,1.594379e-45,1.124836e-14,8.560090e-05,1.681953e-13,1.283599e-18,0.999914,1.253873e-13,2.364274e-18


In [0]:
#tabela de probabilidade por cluster
previsaofinal = pd.concat([comentarios_teste,previsageral.round(3)], axis=1)
previsaofinal.head(1000)

,protocolo,data_manifestacao,municipio_denunciante,municipio_fato,localFato,manifestacao,status_atual,desc_status,sigla,nome,tipo_manifestacao,assunto,sub_assunto,new_manifestacao,0,1,2,3,4,5,6,7
0,2020.0304.134443-14,2020-03-04 13:44:43,NaN,GOIANIA,Setor campinas,A igreja congregação cristã no brasil faz uso ...,55,Finalizado pela ouvidoria do Órgão.,SES,Secretaria de Estado da Saúde,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...",A igreja congregação cristã no brasil faz uso ...,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
1,2020.0313.090346-19,2020-03-13 09:03:46,GOIANIA,GOIANIA,"Rua Sergipe, Qd. B, Lt. 8, Bl-7, Apto. 301, Se...",ILUSTRÍSSIMO (A) SENHOR (A) SUPERINTENDENTE DO...,55,Finalizado pela ouvidoria do Órgão.,SEAD,Secretaria de Estado de Administração (Antiga ...,Solicitação,COVID-19 - Corona Vírus,Informações,ILUSTRÍSSIMO (A) SENHOR (A) SUPERINTENDENTE DO...,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
2,2020.0313.120416-17,2020-03-13 12:04:16,NaN,ANAPOLIS,ANIMA CENTRO HOSPITALAR LTDA CNPJ 18034143000180,Venho por meu deste realizar a denuncia de que...,55,Finalizado pela ouvidoria do Órgão.,SES,Secretaria de Estado da Saúde,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...",Venho por meu deste realizar a denuncia de que...,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
3,2020.0314.132210-80,2020-03-14 13:22:10,NaN,GOIANIA,Cemac Juarez barbosa,Alguém ligado ao governador Ronaldo Caiado pod...,55,Finalizado pela ouvidoria do Órgão.,SES,Secretaria de Estado da Saúde,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...",Alguém ligado ao governador Ronaldo Caiado pod...,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
4,2020.0314.215152-35,2020-03-14 21:51:52,NaN,GOIANIA,Setor Candida de Morais,Elogio com sugestão de ações,55,Finalizado pela ouvidoria do Órgão.,SGG/GOVERNADORIA,Secretaria Geral da Governadoria,Elogio,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...",Elogio com sugestão de ações,0.0,0.01,0.091,0.632,0.172,0.029,0.042,0.023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2020.0323.085913-11,2020-03-23 08:59:13,NaN,GOIANIA,"Av. Santana, 101 - Rodoviário, 74430-420","A empresa está funcionando normalmente, com um...",21,Sob análise do orgão responsável.,SSP,Sec. Estado da Segurança Pública (Policias Civ...,Comunicação,COVID-19 - Corona Vírus,Estabelecimento aberto,A empresa está funcionando normalmente com um...,0.0,0.00,0.281,0.000,0.000,0.718,0.000,0.000
996,2020.0323.085927-98,2020-03-23 08:59:27,NaN,GOIANIA,"Av. Raposo Taváres, 117 - Capuava, Goiânia - G...",Queria denunciar uma empresa de sorvetes em Go...,21,Sob análise do orgão responsável.,SSP,Sec. Estado da Segurança Pública (Policias Civ...,Comunicação,COVID-19 - Corona Vírus,Estabelecimento aberto,Queria denunciar uma empresa de sorvetes em Go...,0.0,0.00,0.995,0.000,0.000,0.005,0.000,0.000
997,2020.0323.085956-70,2020-03-23 08:59:56,NaN,GOIANIA,Bel Hidrodinamica Ind. de Bombas Hidraulicas n...,BOM DIA... GOSTARIA DE ME INFORMAR A RESPEITO...,4,Finalizado pela ouvidoria.,CGE,Controladoria-Geral do Estado de Goiás / Ouvid...,Solicitação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...",BOM DIA GOSTARIA DE ME INFORMAR A RESPEITO...,0.0,0.00,0.000,0.000,0.000,0.022,0.978,0.000
998,2020.0323.090002-38,2020-03-23 09:00:02,NaN,GOIANIA,"Av. C-255, N 400",O Edifício Comercial Eldorado Business Tower l...,21,Sob análise do orgão responsável.,SSP,Sec. Estado da Segurança Pública (Policias Civ...,Comunicação,COVID-19 - Corona Vírus,Estabelecimento aberto,O Edifício Comercial Eldorado Business Tower l...,0.0,0.00,0.000,1.000,0.000,0.000,0.000,0.000


**Exportar** **Modelo** 

In [0]:
import pickle

In [0]:
pickle.dump(modelo, open("/content/drive/My Drive/Colab Notebooks/model_ouvidoria.pkl","wb"))

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer()
X = tfidfVectorizer.fit_transform(corpus).toarray()
tfidfVectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
pickle.dump(tfidfVectorizer, open("/content/drive/My Drive/Colab Notebooks/tfidfVectorizer.pkl","wb"))


In [0]:
model = pickle.load(open("/content/drive/My Drive/Colab Notebooks/model_ouvidoria.pkl", 'rb'))


In [0]:
#teste modelo exportado
testes4 = ['Temos que fechar as indústrias, somos todos vetores de uma possível transmissão']
freq_testes_4 = vectorizer.transform(testes4)


In [0]:
result = model.predict(vectorizer.transform(testes4))
result


array([5])

In [0]:
result_val = model.predict(freq_text_)
result_val

array([5, 5, 5, ..., 2, 5, 5])

In [0]:
previsageral = pd.DataFrame(result_val)

In [0]:
previsageral.round(3)

,0
0,5
1,5
2,5
3,5
4,3
...,...
4198,5
4199,5
4200,2
4201,5


In [0]:
#gerar tabela de cluster
previsaofinal = pd.concat([comentarios_teste,previsageral.round(3)], axis=1)
previsaofinal.head(1000)

,protocolo,data_manifestacao,municipio_denunciante,municipio_fato,localFato,manifestacao,status_atual,desc_status,sigla,nome,tipo_manifestacao,assunto,sub_assunto,new_manifestacao,0
0,2020.0304.134443-14,2020-03-04 13:44:43,NaN,GOIANIA,Setor campinas,A igreja congregação cristã no brasil faz uso ...,55,Finalizado pela ouvidoria do Órgão.,SES,Secretaria de Estado da Saúde,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...",A igreja congregação cristã no brasil faz uso ...,5
1,2020.0313.090346-19,2020-03-13 09:03:46,GOIANIA,GOIANIA,"Rua Sergipe, Qd. B, Lt. 8, Bl-7, Apto. 301, Se...",ILUSTRÍSSIMO (A) SENHOR (A) SUPERINTENDENTE DO...,55,Finalizado pela ouvidoria do Órgão.,SEAD,Secretaria de Estado de Administração (Antiga ...,Solicitação,COVID-19 - Corona Vírus,Informações,ILUSTRÍSSIMO (A) SENHOR (A) SUPERINTENDENTE DO...,5
2,2020.0313.120416-17,2020-03-13 12:04:16,NaN,ANAPOLIS,ANIMA CENTRO HOSPITALAR LTDA CNPJ 18034143000180,Venho por meu deste realizar a denuncia de que...,55,Finalizado pela ouvidoria do Órgão.,SES,Secretaria de Estado da Saúde,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...",Venho por meu deste realizar a denuncia de que...,5
3,2020.0314.132210-80,2020-03-14 13:22:10,NaN,GOIANIA,Cemac Juarez barbosa,Alguém ligado ao governador Ronaldo Caiado pod...,55,Finalizado pela ouvidoria do Órgão.,SES,Secretaria de Estado da Saúde,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...",Alguém ligado ao governador Ronaldo Caiado pod...,5
4,2020.0314.215152-35,2020-03-14 21:51:52,NaN,GOIANIA,Setor Candida de Morais,Elogio com sugestão de ações,55,Finalizado pela ouvidoria do Órgão.,SGG/GOVERNADORIA,Secretaria Geral da Governadoria,Elogio,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...",Elogio com sugestão de ações,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2020.0323.085913-11,2020-03-23 08:59:13,NaN,GOIANIA,"Av. Santana, 101 - Rodoviário, 74430-420","A empresa está funcionando normalmente, com um...",21,Sob análise do orgão responsável.,SSP,Sec. Estado da Segurança Pública (Policias Civ...,Comunicação,COVID-19 - Corona Vírus,Estabelecimento aberto,A empresa está funcionando normalmente com um...,5
996,2020.0323.085927-98,2020-03-23 08:59:27,NaN,GOIANIA,"Av. Raposo Taváres, 117 - Capuava, Goiânia - G...",Queria denunciar uma empresa de sorvetes em Go...,21,Sob análise do orgão responsável.,SSP,Sec. Estado da Segurança Pública (Policias Civ...,Comunicação,COVID-19 - Corona Vírus,Estabelecimento aberto,Queria denunciar uma empresa de sorvetes em Go...,2
997,2020.0323.085956-70,2020-03-23 08:59:56,NaN,GOIANIA,Bel Hidrodinamica Ind. de Bombas Hidraulicas n...,BOM DIA... GOSTARIA DE ME INFORMAR A RESPEITO...,4,Finalizado pela ouvidoria.,CGE,Controladoria-Geral do Estado de Goiás / Ouvid...,Solicitação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...",BOM DIA GOSTARIA DE ME INFORMAR A RESPEITO...,6
998,2020.0323.090002-38,2020-03-23 09:00:02,NaN,GOIANIA,"Av. C-255, N 400",O Edifício Comercial Eldorado Business Tower l...,21,Sob análise do orgão responsável.,SSP,Sec. Estado da Segurança Pública (Policias Civ...,Comunicação,COVID-19 - Corona Vírus,Estabelecimento aberto,O Edifício Comercial Eldorado Business Tower l...,3
